### Importation des bibliothèques

In [51]:
import cv2
import numpy as np
import os

### Création des listes de fichiers

In [52]:
def lister_fichiers(repertoire):
    return os.listdir(repertoire)

### Fonction de comparaison

In [53]:
def comparaison(path1, path2, algo="ORB"):
    img1 = cv2.imread(path1, cv2.IMREAD_COLOR)
    img_grey1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    img2 = cv2.imread(path2, cv2.IMREAD_COLOR)
    img_grey2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    if algo == "ORB":
        feature_detector = cv2.ORB_create(nfeatures=500)
    elif algo == "SIFT":
        feature_detector = cv2.SIFT_create()
    else:
        raise ValueError("L'algorithme spécifié n'est pas pris en charge.")
    
    points1, descripteursImg1 = feature_detector.detectAndCompute(img_grey1, None)
    points2, descripteursImg2 = feature_detector.detectAndCompute(img_grey2, None)

    if descripteursImg1 is None or descripteursImg2 is None:
        return 0  # Aucune correspondance trouvée

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descripteursImg1, descripteursImg2, k=2)

    # Test de ratio de Lowe
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    return len(good_matches)

In [54]:
def tester_algorithmes(pathUser, categories):
    resultats = {"ORB": {}, "SIFT": {}}
    for algo in ["ORB", "SIFT"]:
        DicoComparaison = {list: [] for list in categories.keys()}
        for list, liste_images in categories.items():
            for img in liste_images:
                path = f'IMAGES/Imagettes/{list.lower()}/{img}'
                score = comparaison(pathUser, path, algo=algo)
                DicoComparaison[list].append(score)
        moyennes = {list: np.mean(scores) for list, scores in DicoComparaison.items()}
        resultats[algo] = moyennes
    return resultats


In [55]:
# On charge les images de référence
categories = {
    "Avions": lister_fichiers('IMAGES/Imagettes/avions'),
    "Drones": lister_fichiers('IMAGES/Imagettes/drones'),
    "Hélicoptères": lister_fichiers('IMAGES/Imagettes/hélicoptères'),
    "Missiles": lister_fichiers('IMAGES/Imagettes/missiles'),
    "Oiseaux": lister_fichiers('IMAGES/Imagettes/oiseaux')
}

# On demande l'image utilisateur
pathUser = input("Rentrez le path de votre image (Exemple : IMAGES/Images/missile.jpg) : ")

# On test ORB et SIFT
resultats = tester_algorithmes(pathUser, categories)

# On compare les performances
meilleurs_scores = {algo: max(scores.values()) for algo, scores in resultats.items()}
meilleur_algo = max(meilleurs_scores, key=meilleurs_scores.get)

print("\nRésultats par algorithme :")
for algo, scores in resultats.items():
    print(f"\n{algo}:")
    for cat, moy in scores.items():
        print(f"  {cat}: {moy}")

print(f"\nMeilleur algorithme : {meilleur_algo}")
print(f"Catégorie prédite : {max(resultats[meilleur_algo], key=resultats[meilleur_algo].get)}")



Résultats par algorithme :

ORB:
  Avions: 2.4
  Drones: 3.4
  Hélicoptères: 4.3
  Missiles: 3.3
  Oiseaux: 2.6

SIFT:
  Avions: 18.5
  Drones: 14.7
  Hélicoptères: 14.9
  Missiles: 15.2
  Oiseaux: 20.1

Meilleur algorithme : SIFT
Catégorie prédite : Oiseaux
